In [1]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from env_vars import sentinel_username,sentinel_password
import glob
import pandas as pd
import os
pd.set_option('display.max_colwidth', -1)

C:\Users\David\.virtualenvs\canopy-3IjoYI-i\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
api = SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")

In [3]:
help(api.query)

Help on method query in module sentinelsat.sentinel:

query(area=None, date=None, raw=None, area_relation='Intersects', order_by=None, limit=None, offset=0, **keywords) method of sentinelsat.sentinel.SentinelAPI instance
    Query the OpenSearch API with the coordinates of an area, a date interval
    and any other search keywords accepted by the API.
    
    Parameters
    ----------
    area : str, optional
        The area of interest formatted as a Well-Known Text string.
    date : tuple of (str or datetime) or str, optional
        A time interval filter based on the Sensing Start Time of the products.
        Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW").
        The timestamps can be either a Python datetime or a string in one of the
        following formats:
    
            - yyyyMMdd
            - yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)
            - yyyy-MM-ddThh:mm:ssZ
            - NOW
            - NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
            - NOW+<n

In [ ]:
# download single scene by known product id
# api.download("3f09e58e-b893-46fa-ad05-6bf1f5b4b257")

# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))
products = api.query(footprint,
                     date=('20150601', date(2015, 12, 31)),
                     area_relation = "IsWithin",
                     raw='1C',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))


# # # convert to Pandas DataFrame
products_df = api.to_dataframe(products)

        Additional query text that will be appended to the query.
    area_relation : {'Intersects', 'Contains', 'IsWithin'}, optional
        What relation to use for testing the AOI. Case insensitive.
    
            - Intersects: true if the AOI and the footprint intersect (default)
            - Contains: true if the AOI is inside the footprint
            - IsWithin: true if the footprint is inside the AOI

In [5]:
products_df.shape # should be 418, 35

(418, 35)

In [6]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'datatakesensingstart', 'beginposition', 'endposition', 'ingestiondate',
       'orbitnumber', 'relativeorbitnumber', 'cloudcoverpercentage',
       'gmlfootprint', 'format', 'instrumentshortname',
       'sensoroperationalmode', 'instrumentname', 'footprint', 's2datatakeid',
       'platformidentifier', 'orbitdirection', 'platformserialidentifier',
       'processingbaseline', 'processinglevel', 'producttype', 'platformname',
       'size', 'tileid', 'hv_order_tileid', 'filename', 'identifier', 'uuid',
       'level1cpdiidentifier', 'granuleidentifier', 'datastripidentifier'],
      dtype='object')

In [7]:
products_df_2 = products_df.drop_duplicates(subset='tileid')

In [8]:
len(products_df_2)

158

In [9]:
online = []
for product_id in products_df_2.index:
    if api.is_online(product_id):
#         print('Product {} is online. Adding to list.'.format(product_id))
        online.append(product_id)
    else:
        continue
print("total online products:",len(online))

total online products: 90


In [10]:
products_df_3 = products_df_2.loc[online]

In [11]:
len(products_df_3)

90

In [13]:
downloaded_files = [s.split(".")[0] for s in os.listdir("./data/Sentinel_Data/2015")]

In [16]:
products_df_4 = products_df_3[~products_df_3["title"].isin(downloaded_files)]

In [15]:
# products_df_4["product_id"] = products_df_4.index

In [17]:
from collections import OrderedDict

# Get the unordered dictionary
# unordered_products_dict = products_df_4.set_index('product_id').T.to_dict()

 # Then order it
# ordered_dict = OrderedDict((k,unordered_products_dict.get(k)) for k in products_df_4.product_id)

In [19]:
unordered_products_dict = products_df_4.T.to_dict()

ordered_dict = OrderedDict((k, unordered_products_dict.get(k)) for k in products_df_4.index)

In [20]:
products_df_4.head(1)

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,platformname,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158,https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/$value,https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/,https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/Products('Quicklook')/$value,"Date: 2015-07-08T08:10:16.027Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 152.40 MB",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:23:28.140,218,...,Sentinel-2,152.40 MB,35NPA,NA35P,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158.SAFE,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158,55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A000218_T35NPA_N02.04,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A000218_T35NPA_N02.04,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S20150708T082158_N02.04


In [21]:
ordered_dict['55797ce2-d2a1-4e15-826c-75a4f421a21a']

{'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158',
 'link': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/$value",
 'link_alternative': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/",
 'link_icon': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/Products('Quicklook')/$value",
 'summary': 'Date: 2015-07-08T08:10:16.027Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 152.40 MB',
 'datatakesensingstart': Timestamp('2015-07-08 08:10:16.027000'),
 'beginposition': Timestamp('2015-07-08 08:10:16.027000'),
 'endposition': Timestamp('2015-07-08 08:10:16.027000'),
 'ingestiondate': Timestamp('2019-02-04 20:23:28.140000'),
 'orbitnumber': 218,
 'relativeorbitnumber': 78,
 'cloudcoverpercentage': 1.3745,
 'gmlfootprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" xmlns:gml="http://www.opengis.net/

In [22]:
products['55797ce2-d2a1-4e15-826c-75a4f421a21a']

{'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158',
 'link': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/$value",
 'link_alternative': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/",
 'link_icon': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/Products('Quicklook')/$value",
 'summary': 'Date: 2015-07-08T08:10:16.027Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 152.40 MB',
 'datatakesensingstart': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'beginposition': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'endposition': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'ingestiondate': datetime.datetime(2019, 2, 4, 20, 23, 28, 140000),
 'orbitnumber': 218,
 'relativeorbitnumber': 78,
 'cloudcoverpercentage': 1.3745,
 'gmlfootprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" 

In [23]:
# download all results from the search
api.download_all(ordered_dict,directory_path="./data/Sentinel_Data/2015/")

Downloading:   0%|                                                                          | 0.00/160M [00:00<?, ?B/s]
Downloading:   0%|                                                                          | 0.00/322M [00:00<?, ?B/s]
Downloading:   0%|▏                                                                 | 1.05M/322M [00:01<06:10, 868kB/s]
Downloading:   1%|▍                                                                | 2.10M/322M [00:01<04:43, 1.13MB/s]
Downloading:   1%|▋                                                                | 3.15M/322M [00:01<03:30, 1.51MB/s]
Downloading:   1%|▊                                                                | 4.19M/322M [00:01<02:42, 1.95MB/s]
Downloading:   2%|█                                                                | 5.24M/322M [00:02<02:31, 2.10MB/s]
Downloading:   2%|█▎                                                               | 6.29M/322M [00:02<01:55, 2.73MB/s]
Downloading:   2%|█▍                    

Downloading:  16%|██████████▋                                                      | 26.2M/160M [00:29<00:45, 2.93MB/s]
Downloading:  17%|███████████                                                      | 27.3M/160M [00:29<00:45, 2.94MB/s]
Downloading:  18%|███████████▉                                                     | 29.4M/160M [00:30<00:42, 3.08MB/s]
Downloading:  19%|████████████▎                                                    | 30.4M/160M [00:30<00:38, 3.41MB/s]
Downloading:  20%|████████████▊                                                    | 31.5M/160M [00:30<00:36, 3.52MB/s]
Downloading:  20%|█████████████▏                                                   | 32.5M/160M [00:31<00:36, 3.50MB/s]
Downloading:  21%|█████████████▋                                                   | 33.6M/160M [00:31<00:33, 3.76MB/s]
Downloading:  22%|██████████████                                                   | 34.6M/160M [00:31<00:36, 3.48MB/s]
Downloading:  23%|██████████████▉       

Downloading:   3%|██                                                               | 15.7M/493M [00:05<02:41, 2.96MB/s]
Downloading:  87%|█████████████████████████████████████████████████████████▎        | 280M/322M [01:14<00:16, 2.61MB/s]
Downloading:   4%|██▍                                                              | 18.9M/493M [00:06<02:40, 2.96MB/s]
Downloading:   4%|██▋                                                              | 19.9M/493M [00:06<02:40, 2.95MB/s]
Downloading:   4%|██▉                                                              | 22.0M/493M [00:07<02:37, 2.98MB/s]
Downloading:   5%|███▏                                                             | 24.1M/493M [00:07<02:26, 3.21MB/s]
Downloading:   5%|███▍                                                             | 26.2M/493M [00:08<02:15, 3.44MB/s]
Downloading:   6%|███▌                                                             | 27.3M/493M [00:08<02:30, 3.09MB/s]
Downloading:   6%|███▋                  

Downloading:  34%|██████████████████████▊                                           | 170M/493M [00:54<01:01, 5.28MB/s]
Downloading:  35%|███████████████████████▏                                          | 173M/493M [00:55<01:15, 4.24MB/s]
Downloading:  36%|███████████████████████▍                                          | 175M/493M [00:56<01:20, 3.95MB/s]
Downloading:  36%|███████████████████████▌                                          | 176M/493M [00:56<01:16, 4.12MB/s]
Downloading:  36%|███████████████████████▉                                          | 178M/493M [00:56<01:14, 4.22MB/s]
Downloading:  37%|████████████████████████▏                                         | 180M/493M [00:57<01:11, 4.38MB/s]
Downloading:  37%|████████████████████████▎                                         | 181M/493M [00:57<01:18, 3.98MB/s]
Downloading:  37%|████████████████████████▌                                         | 184M/493M [00:58<01:21, 3.79MB/s]
Downloading:  37%|██████████████████████

Downloading:  63%|█████████████████████████████████████████▋                        | 311M/493M [01:39<00:51, 3.54MB/s]
Downloading:  64%|██████████████████████████████████████████                        | 314M/493M [01:39<00:52, 3.44MB/s]
Downloading:  64%|██████████████████████████████████████████▎                       | 316M/493M [01:40<00:50, 3.52MB/s]
Downloading:  64%|██████████████████████████████████████████▌                       | 318M/493M [01:41<00:54, 3.21MB/s]
Downloading:  65%|██████████████████████████████████████████▊                       | 320M/493M [01:41<00:52, 3.32MB/s]
Downloading:  65%|███████████████████████████████████████████▏                      | 322M/493M [01:42<00:50, 3.40MB/s]
Downloading:  66%|███████████████████████████████████████████▍                      | 324M/493M [01:42<00:51, 3.29MB/s]
Downloading:  66%|███████████████████████████████████████████▋                      | 326M/493M [01:43<00:47, 3.48MB/s]
Downloading:  67%|██████████████████████

...

Downloading:  68%|████████████████████████████████████████████▉                     | 560M/821M [03:06<01:13, 3.55MB/s]
Downloading:  69%|█████████████████████████████████████████████▏                    | 563M/821M [03:06<01:07, 3.85MB/s]
Downloading:  69%|█████████████████████████████████████████████▍                    | 565M/821M [03:07<01:04, 3.99MB/s]
Downloading:  69%|█████████████████████████████████████████████▋                    | 568M/821M [03:08<00:58, 4.33MB/s]
Downloading:  70%|█████████████████████████████████████████████▉                    | 571M/821M [03:08<00:56, 4.40MB/s]
Downloading:  70%|██████████████████████████████████████████████                    | 574M/821M [03:09<00:59, 4.17MB/s]
Downloading:  70%|██████████████████████████████████████████████▎                   | 577M/821M [03:10<01:00, 4.07MB/s]
Downloading:  71%|██████████████████████████████████████████████▌                   | 580M/821M [03:10<00:57, 4.18MB/s]
Downloading:  71%|██████████████████████

Downloading:  91%|████████████████████████████████████████████████████████████▎     | 751M/821M [04:04<00:27, 2.54MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▍     | 752M/821M [04:05<00:27, 2.51MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▍     | 753M/821M [04:05<00:26, 2.61MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▌     | 754M/821M [04:05<00:24, 2.79MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▋     | 755M/821M [04:06<00:23, 2.82MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▋     | 756M/821M [04:06<00:22, 2.90MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▊     | 757M/821M [04:06<00:22, 2.86MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▉     | 758M/821M [04:07<00:21, 2.98MB/s]
Downloading:  92%|██████████████████████

Downloading:  25%|████████████████▍                                                | 54.5M/216M [00:12<00:33, 4.78MB/s]
Downloading:   1%|▍                                                                | 3.15M/481M [00:02<07:03, 1.13MB/s]
Downloading:  26%|█████████████████                                                | 56.6M/216M [00:13<00:33, 4.70MB/s]
Downloading:  27%|█████████████████▍                                               | 57.7M/216M [00:13<00:37, 4.22MB/s]
Downloading:   1%|▌                                                                | 4.19M/481M [00:03<06:22, 1.25MB/s]
Downloading:  28%|██████████████████                                               | 59.8M/216M [00:14<00:39, 3.98MB/s]
Downloading:   1%|▋                                                                | 5.24M/481M [00:03<05:47, 1.37MB/s]
Downloading:  29%|██████████████████▋                                              | 61.9M/216M [00:14<00:36, 4.25MB/s]
Downloading:   1%|▊                     

Downloading:  21%|██████████████                                                    | 103M/481M [00:46<03:13, 1.95MB/s]
Downloading:  22%|██████████████▏                                                   | 104M/481M [00:46<03:13, 1.95MB/s]
Downloading:  90%|███████████████████████████████████████████████████████████▋      | 195M/216M [00:57<00:07, 2.92MB/s]
Downloading:  22%|██████████████▍                                                   | 105M/481M [00:47<03:11, 1.96MB/s]
Downloading:  22%|██████████████▌                                                   | 106M/481M [00:47<03:14, 1.93MB/s]
Downloading:  92%|████████████████████████████████████████████████████████████▋     | 198M/216M [00:58<00:06, 2.87MB/s]
Downloading:  22%|██████████████▋                                                   | 107M/481M [00:48<03:23, 1.84MB/s]
Downloading:  22%|██████████████▊                                                   | 108M/481M [00:49<03:22, 1.84MB/s]
Downloading:  93%|██████████████████████

({'55797ce2-d2a1-4e15-826c-75a4f421a21a': {'id': '55797ce2-d2a1-4e15-826c-75a4f421a21a',
   'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_20150708T082158',
   'size': 159846560,
   'md5': '5B73E054732161A4CEDB22E1EF884814',
   'date': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
   'footprint': 'POLYGON((28.605302845400384 -0.088445084557708,28.608382192445568 -0.074464255781272,28.6154561639182 -0.042179567472391,28.641109876895488 0.074343239947912,28.674107056607248 0.222756251914671,28.70668682339168 0.371241921146265,28.739746530121764 0.519535058725677,28.763365823790085 0.628437155659785,28.771969575886352 0.667940523831296,28.80455203158687 0.816506174667484,28.823819239718045 0.904442244915831,28.885310549700094 0.904418497286558,28.885079391653544 -0.08843451908062,28.605302845400384 -0.088445084557708))',
   'url': "https://scihub.copernicus.eu/apihub/odata/v1/Products('55797ce2-d2a1-4e15-826c-75a4f421a21a')/$value",
   'Online': True,
   'Creation Date': dateti

In [57]:
help(api.download_all)

Help on method download_all in module sentinelsat.sentinel:

download_all(products, directory_path='.', max_attempts=10, checksum=True, n_concurrent_dl=2, lta_retry_delay=600) method of sentinelsat.sentinel.SentinelAPI instance
    Download a list of products.
    
    Takes a list of product IDs as input. This means that the return value of query() can be
    passed directly to this method.
    
    File names on the server are used for the downloaded files, e.g.
    "S1A_EW_GRDH_1SDH_20141003T003840_20141003T003920_002658_002F54_4DD1.zip".
    
    In case of interruptions or other exceptions, downloading will restart from where it left
    off. Downloading is attempted at most max_attempts times to avoid getting stuck with
    unrecoverable errors.
    
    Parameters
    ----------
    products : list
        List of product IDs
    directory_path : string
        Directory where the downloaded files will be downloaded
    max_attempts : int, optional
        Number of allowed retr

In [1]:
def download_tcis(api, footprint, date_start, date_end, zip_folder, tci_folder,
                  area_relation='IsWithin', raw='IC',
                  platformname='Sentinel-2', cloudcoverpercentage=(0,10)):
    
    products = api.query(footprint,
                         date=(date_start, date_end),
                         area_relation=area_relation,
                         raw=raw,
                         platformname=platformname,
                         cloudcoverpercentage=cloudcoverpercentage)
    
    print(f'Got {len(products)} products')
    
    products_df = api.to_dataframe(products)

    products_df = products_df.drop_duplicates(subset='tileid')

    online = [product_id for product_id in products_df.index if api.isonline(product_id)]
    products_df = products_df.loc[online]
    
    unordered_products_dict = products_df.T.to_dict()
    ordered_dict = OrderedDict((k, unordered_products_dict.get(k)) for k in products_df.index)

    print(f'Downloading {len(ordered_dict)} files')
    api.download_all(ordered_dict, directory_path=zip_folder)
    print('Download finished')
    
    zip_files = glob.glob(zip_folder + '*.zip')
    for zip_file in zip_files:
        with ZipFile(zip_file) as z:
            ret = z.testzip()
            if ret is not None:
                print("First bad file in zip: %s" % ret)

            full = z.namelist()
            file_path = [s for s in full if file_string in s][0]
            filename = file_path.split("/")[-1]
            uri = tci_folder
            with open(uri + filename, 'wb') as f:
                f.write(z.read(file_path))
                
        os.remove(zip_file)
        
    print('All TCIs extracted')

In [1]:
import glob
import os
import pandas as pd
import sys
from collections import OrderedDict
from datetime import date
from env_vars import sentinel_username,sentinel_password
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from zipfile import ZipFile

In [18]:
api = SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")

In [17]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))
date_start = '20160101'
date_end = '20160630'
zip_folder = './data/Sentinel_Data/2016/'
tci_folder = './data/Sentinel_data/TCI/2016/'

In [18]:
download_tcis(api, footprint, date_start, date_end, zip_folder, tci_folder)

Got products
Start download


StopIteration: 

In [19]:
def get_ordered_dict(api, footprint, date_start, date_end, zip_folder, tci_folder,
                  area_relation='IsWithin', raw='IC',
                  platformname='Sentinel-2', cloudcoverpercentage=(0,10)):
    
    products = api.query(footprint,
                         date=(date_start, date_end),
                         area_relation=area_relation,
                         raw=raw,
                         platformname=platformname,
                         cloudcoverpercentage=cloudcoverpercentage)
    
    print('Got products')
    
    products_df = api.to_dataframe(products)

    products_df = products_df.drop_duplicates(subset='tileid')

    online = [product_id for product_id in products_df.index if api.isonline(product_id)]
    products_df = products_df.loc[online]
    
    unordered_products_dict = products_df.T.to_dict()
    ordered_dict = OrderedDict((k, unordered_products_dict.get(k)) for k in products_df.index)
    
    return products, ordered_dict

In [20]:
products, ordered_dict = get_ordered_dict(api, footprint, date_start, date_end, zip_folder, tci_folder)

Got products


In [23]:
ordered_dict

OrderedDict()

In [28]:
products = api.query(footprint,
                     date=('20160101', date(2016, 6, 30)),
                     area_relation = "IsWithin",
                     raw='1C',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

Querying products: 100%|████████████████████████████████████████████████████| 1045/1045 [01:43<00:00, 10.09 products/s]


In [29]:
len(products)

1045

In [4]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))
date_start = '20160101'
date_end = date(2016, 12, 31)
zip_folder = './data/Sentinel_Data/2016/'
tci_folder = './data/Sentinel_data/TCI/2016/'

In [5]:
download_tcis(api, footprint, date_start, date_end, zip_folder, tci_folder)

Got 0 products


StopIteration: 

In [6]:
products = api.query(footprint,
                     date=('20160101', date(2016, 12, 31)),
                     area_relation = "IsWithin",
                     raw='1C',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

Querying products: 100%|████████████████████████████████████████████████████| 1864/1864 [04:25<00:00,  7.03 products/s]


In [7]:
len(products)

1864

In [2]:
def get_products(api, footprint, date_start, date_end,
                 area='IsWithin', raw='1C',
                 platform='Sentinel-2', cloudcover=(0,10)):
    
    return api.query(footprint,
                     date=(date_start, date_end),
                     area_relation=area,
                     raw=raw,
                     platformname=platform,
                     cloudcoverpercentage=cloudcover)

In [10]:
products = get_products(api, footprint, '20160101', '20160630')

Querying products: 100%|████████████████████████████████████████████████████| 1045/1045 [02:07<00:00,  8.21 products/s]


In [11]:
len(products)

1045

In [3]:
def clean_products(api, products):

    products_df = api.to_dataframe(products)

    online = [product_id for product_id in products_df.index if api.is_online(product_id)]
    products_df = products_df.loc[online]
    
    products_df = products_df.drop_duplicates(subset='tileid')
    
    unordered_products_dict = products_df.T.to_dict()
    
    return OrderedDict((k, unordered_products_dict.get(k)) for k in products_df.index)

In [16]:
ordered_dict = clean_products(api, products)

len(ordered_dict)

1

In [21]:
products = get_products(api, footprint, date(2016, 7, 1), date(2016, 12, 31))
ordered_dict = clean_products(api, products)

len(ordered_dict)

Querying products: 100%|██████████████████████████████████████████████████████| 816/816 [01:31<00:00,  8.91 products/s]


7

In [4]:
def get_api():
    
    return SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")

In [26]:
api = get_api()
products = get_products(api, footprint, date(2017, 1, 1), date(2017, 1, 31))
ordered_dict = clean_products(api, products)

len(ordered_dict)

Querying products: 100%|██████████████████████████████████████████████████████| 136/136 [00:08<00:00, 15.92 products/s]


0

In [5]:
def number_of_prods(footprint, date_start, date_end):
    
    api = get_api()
    products = get_products(api, footprint, date_start, date_end)

    return len(clean_products(api, products))

In [8]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))

prods = {}

years = [2017, 2018, 2019, 2020]
months_dict = {
    'Jan': (1, 31),
    'Feb': (2, 28),
    'Mar': (3, 31),
    'Apr': (4, 30),
    'May': (5, 31),
    'Jun': (6, 30),
    'Jul': (7, 31),
    'Aug': (8, 31),
    'Sep': (9, 30),
    'Oct': (10, 31),
    'Nov': (11, 30),
    'Dec': (12, 31)
}

for year in years:
    for month in months_dict.keys():
        print(f'Start {month} {year}')
        month_num = months_dict[month][0]
        day = months_dict[month][1]
        
        if year == 2020 and month_num > 6:
            prods[f'{year} {month}'] = 'N/A'
        
        else:
            start = date(year, month_num, 1)
            end = date(year, month_num, day)
        
            num_of_prods = number_of_prods(footprint, start, end)
        
            prods[f'{year} {month}'] = num_of_prods
        print(f'End {month} {year}')
        
prods

Start Jan 2017


Querying products: 100%|██████████████████████████████████████████████████████| 136/136 [00:07<00:00, 18.47 products/s]


End Jan 2017
Start Feb 2017


Querying products: 100%|██████████████████████████████████████████████████████| 151/151 [00:05<00:00, 26.14 products/s]


End Feb 2017
Start Mar 2017


...

Querying products: 100%|██████████████████████████████████████████████████████| 317/317 [00:24<00:00, 12.97 products/s]


End May 2020
Start Jun 2020


Querying products: 100%|██████████████████████████████████████████████████████| 343/343 [00:29<00:00, 11.53 products/s]


End Jun 2020
Start Jul 2020
End Jul 2020
Start Aug 2020
End Aug 2020
Start Sep 2020
End Sep 2020
Start Oct 2020
End Oct 2020
Start Nov 2020
End Nov 2020
Start Dec 2020
End Dec 2020


{'2017 Jan': 0,
 '2017 Feb': 0,
 '2017 Mar': 0,
 '2017 Apr': 0,
 '2017 May': 0,
 '2017 Jun': 0,
 '2017 Jul': 27,
 '2017 Aug': 0,
 '2017 Sep': 0,
 '2017 Oct': 0,
 '2017 Nov': 0,
 '2017 Dec': 0,
 '2018 Jan': 0,
 '2018 Feb': 0,
 '2018 Mar': 0,
 '2018 Apr': 0,
 '2018 May': 0,
 '2018 Jun': 0,
 '2018 Jul': 0,
 '2018 Aug': 0,
 '2018 Sep': 0,
 '2018 Oct': 0,
 '2018 Nov': 0,
 '2018 Dec': 0,
 '2019 Jan': 0,
 '2019 Feb': 0,
 '2019 Mar': 0,
 '2019 Apr': 0,
 '2019 May': 0,
 '2019 Jun': 0,
 '2019 Jul': 137,
 '2019 Aug': 130,
 '2019 Sep': 111,
 '2019 Oct': 64,
 '2019 Nov': 98,
 '2019 Dec': 134,
 '2020 Jan': 176,
 '2020 Feb': 181,
 '2020 Mar': 135,
 '2020 Apr': 140,
 '2020 May': 139,
 '2020 Jun': 124,
 '2020 Jul': 'N/A',
 '2020 Aug': 'N/A',
 '2020 Sep': 'N/A',
 '2020 Oct': 'N/A',
 '2020 Nov': 'N/A',
 '2020 Dec': 'N/A'}

In [9]:
prods

{'2017 Jan': 0,
 '2017 Feb': 0,
 '2017 Mar': 0,
 '2017 Apr': 0,
 '2017 May': 0,
 '2017 Jun': 0,
 '2017 Jul': 27,
 '2017 Aug': 0,
 '2017 Sep': 0,
 '2017 Oct': 0,
 '2017 Nov': 0,
 '2017 Dec': 0,
 '2018 Jan': 0,
 '2018 Feb': 0,
 '2018 Mar': 0,
 '2018 Apr': 0,
 '2018 May': 0,
 '2018 Jun': 0,
 '2018 Jul': 0,
 '2018 Aug': 0,
 '2018 Sep': 0,
 '2018 Oct': 0,
 '2018 Nov': 0,
 '2018 Dec': 0,
 '2019 Jan': 0,
 '2019 Feb': 0,
 '2019 Mar': 0,
 '2019 Apr': 0,
 '2019 May': 0,
 '2019 Jun': 0,
 '2019 Jul': 137,
 '2019 Aug': 130,
 '2019 Sep': 111,
 '2019 Oct': 64,
 '2019 Nov': 98,
 '2019 Dec': 134,
 '2020 Jan': 176,
 '2020 Feb': 181,
 '2020 Mar': 135,
 '2020 Apr': 140,
 '2020 May': 139,
 '2020 Jun': 124,
 '2020 Jul': 'N/A',
 '2020 Aug': 'N/A',
 '2020 Sep': 'N/A',
 '2020 Oct': 'N/A',
 '2020 Nov': 'N/A',
 '2020 Dec': 'N/A'}